# Deep Convolutional Neural Network in Keras

Bulding a Deep Convolutional Neural Network to classify MNIST digits.

#### Set seed for reproducibility

In [1]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten, MaxPooling2D, Conv2D, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, LearningRateScheduler

from sklearn.model_selection import train_test_split

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


#### Load data

In [3]:
n_classes = 10

raw_data = np.loadtxt('kaggle/datasets/train.csv', skiprows=1, dtype='int', delimiter=',')
X_train, X_val, y_train, y_val = train_test_split(
    raw_data[:,1:], raw_data[:,0], test_size = 0.1)

X_train = X_train.reshape(-1, 28, 28, 1)
X_val = X_val.reshape(-1, 28, 28, 1)

X_train = X_train.astype("float32")/255.
X_val = X_val.astype("float32")/255.

y_train = keras.utils.to_categorical(y_train, n_classes)
y_val = keras.utils.to_categorical(y_val, n_classes)


#### Define Custom Activation Function

In [4]:
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

def relus(Z):
    e_param = 3.8
    pi = K.variable((3.14))
    m = e_param + (K.sigmoid(K.sin(Z)) - K.sigmoid(K.cos(Z)) * K.exp(K.sqrt(pi)))
    A = K.maximum(m, Z)
    return A

get_custom_objects().update({'ReLU_s': Activation(relus)})

#### Design Neural Network architecture

In [5]:
model = Sequential()

model.add(Conv2D(16, 4, padding = 'same', activation ='ReLU_s', input_shape = (28, 28, 1)))
model.add(Conv2D(16, 4, padding = 'same', activation ='ReLU_s'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(20, 7, padding = 'same', activation ='ReLU_s'))
model.add(Conv2D(20, 7, padding = 'same', activation ='ReLU_s'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(28, 2, padding = 'same', activation ='ReLU_s'))
model.add(Conv2D(28, 2, activation ='ReLU_s'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation = "ReLU_s"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        272       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 16)        4112      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 20)        15700     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 20)        19620     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 20)          0         
__________

#### Data Augmentation

In [7]:
datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

#### Learning Rate Scheduler

In [8]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

#### Configure model

In [9]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#### TensorBoard

In [10]:
tensorboard = TensorBoard("logs/lenet-cnn-mnist-ReLUs-no-augmentation-30-epochs-new-layer")

#### Train!

In [11]:
# hist = model.fit_generator(datagen.flow(X_train, y_train, batch_size = 64),
#                            steps_per_epoch = 300,
#                            epochs = 65,
#                            verbose = 1,
#                            validation_data = (X_val[:400,:], y_val[:400,:]),
#                            callbacks = [tensorboard])

hist = model.fit(X_train, y_train, batch_size = 32,
                           epochs = 30,
                           verbose = 1,
                           validation_split = 0.1,
                           callbacks=[annealer, tensorboard])

Train on 34020 samples, validate on 3780 samples
Epoch 1/30
34020/34020 [==============================] - 150s - loss: 0.2878 - acc: 0.9044 - val_loss: 0.0682 - val_acc: 0.9791
Epoch 2/30
34020/34020 [==============================] - 145s - loss: 0.0828 - acc: 0.9756 - val_loss: 0.0499 - val_acc: 0.9862
Epoch 3/30
34020/34020 [==============================] - 147s - loss: 0.0588 - acc: 0.9824 - val_loss: 0.0332 - val_acc: 0.9910
Epoch 4/30
34020/34020 [==============================] - 145s - loss: 0.0487 - acc: 0.9855 - val_loss: 0.0332 - val_acc: 0.9905
Epoch 5/30
34020/34020 [==============================] - 151s - loss: 0.0409 - acc: 0.9883 - val_loss: 0.0284 - val_acc: 0.9910
Epoch 6/30
34020/34020 [==============================] - 149s - loss: 0.0370 - acc: 0.9894 - val_loss: 0.0270 - val_acc: 0.9929
Epoch 7/30
34020/34020 [==============================] - 152s - loss: 0.0299 - acc: 0.9907 - val_loss: 0.0328 - val_acc: 0.9915
Epoch 8/30
34020/34020 [========================

#### Test Final Accuracy

In [12]:
final_loss, final_acc = model.evaluate(X_val, y_val, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

4192/4200 [============================>.] - ETA: 0sFinal loss: 0.0283, final accuracy: 0.9931


#### Save Submission

In [14]:
X_test = np.loadtxt('kaggle/datasets/test.csv', skiprows=1, dtype='int', delimiter=',')
X_test = X_test.reshape(28000, 28, 28, 1).astype('float32') / 255.

predictions = model.predict(X_test, verbose = 2)
predictions = np.argmax(predictions, axis = 1)

pd.DataFrame({"ImageId": list(range(1, len(predictions) + 1)), "Label": predictions}).to_csv('kaggle/results/cnn/submission-ReLUs-no-augmentation-30-epochs-new-layer.csv', index = False, header = True)